In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#필요한 lib으로 한번만 하고, jupyter환경에선 다신 실행 안하셔도되고, colab에선 실행할 때마다 해주심 됩니다.
!pip install catboost
!pip install lightgbm
!pip install ngboost
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.2/349.2 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 12.7 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=aa44884f8a5d6467019c209ebd4ca0419a390d9d0599f882824ff3aa0027066c
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 5.4 MB/s eta 0:00:00


In [3]:
# Essentials
import numpy as np
import pandas as pd
import random
import time
import gc
import os
from datetime import datetime

# Plots
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
figure(num=None, figsize=(20, 10), dpi=80, facecolor='w', edgecolor='k')
import seaborn as sns
# matplotlib 의 기본 scheme 말고
# seaborn scheme 을 세팅하여, 일일이 graph의 font size 지정할 필요 없이
# seaborn 의 font_scale 을 사용하는 것을 추천드립니다.
plt.style.use('seaborn-v0_8') # 이 위에 코드를 실행하면 여러 테마가 있으니 마음껏 바꾸셔도 됩니다.
sns.set(font_scale=2.5)
from matplotlib.pylab import rcParams
##set up the parameters
rcParams['figure.figsize'] = 80,60

# Models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from ngboost import NGBClassifier
import lightgbm
import xgboost as xgb
import ngboost
import catboost

from xgboost import plot_tree

# Stats
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax


# Misc
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.metrics import confusion_matrix


from dateutil import tz

import shap
import skimage


# Ignore useless warnings
import warnings
warnings.filterwarnings(action="ignore")
pd.options.display.max_seq_items = 8000
pd.options.display.max_rows = 8000
pd.options.display.max_columns = 400
pd.set_option('display.max_colwidth', None)

<Figure size 1600x800 with 0 Axes>

In [4]:
path = '/content/drive/MyDrive/Colab Notebooks/model/4.11_obj_modi.csv'
df = pd.read_csv(path)
df

,Unnamed: 0,installment,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,pub_rec,pub_rec_bankruptcies,annual_inc,dti,funded_amnt_inv,grade,int_rate,loan_amnt,mort_acc,num_accts_ever_120_pd,issue_d_year,issue_d_month,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 60 months,addr_state_AL,addr_state_AR,addr_state_AZ,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_IA,addr_state_ID,addr_state_IL,addr_state_IN,addr_state_KS,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_ND,addr_state_NE,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,fico_range_mean,target
0,1,680.82,126.0,277.0,0.0,0.0,110000.0,34.70,23200.000000,3,24.99,23200.0,2.0,0.0,2016,7,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,672.0,1
1,3,494.55,134.0,400.0,0.0,0.0,65000.0,17.74,16000.000000,7,7.07,16000.0,3.0,0.0,2017,10,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,822.0,1
2,4,146.10,226.0,130.0,2.0,2.0,50000.0,6.99,4500.000000,6,10.42,4500.0,1.0,0.0,2017,5,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,702.0,0
3,5,645.25,149.0,190.0,0.0,0.0,60000.0,28.94,20000.000000,6,9.99,20000.0,1.0,5.0,2015,9,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,677.0,0
4,6,482.17,142.0,206.0,0.0,0.0,97523.0,10.31,18999.739941,4,17.97,19000.0,3.0,0.0,2019,5,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,737.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1071205,1534129,662.45,232.0,144.0,0.0,0.0,60000.0,20.02,26450.000000,5,17.27,26500.0,1.0,0.0,2013,2,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,697.0,0
1071206,1534131,133.78,151.0,36.0,0.0,0.0,33800.0,19.18,3600.000000,3,19.99,3600.0,1.0,0.0,2014,3,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,667.0,1
1071207,1534132,583.89,125.0,216.0,0.0,0.0,71000.0,23.92,18875.000000,7,6.68,19000.0,5.0,0.0,2015,4,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,737.0,0
1071208,1534133,329.65,25.0,99.0,0.0,0.0,44000.0,12.85,9600.000000,5,14.33,9600.0,1.0,0.0,2013,5,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,692.0,0


In [5]:
# 결측치 있나 확인
df.isnull().sum()

Unnamed: 0                    0
installment                   0
mo_sin_old_il_acct            0
mo_sin_old_rev_tl_op          0
pub_rec                       0
pub_rec_bankruptcies          0
annual_inc                    0
dti                           0
funded_amnt_inv               0
grade                         0
int_rate                      0
loan_amnt                     0
mort_acc                      0
num_accts_ever_120_pd         0
issue_d_year                  0
issue_d_month                 0
home_ownership_MORTGAGE       0
home_ownership_NONE           0
home_ownership_OTHER          0
home_ownership_OWN            0
home_ownership_RENT           0
purpose_credit_card           0
purpose_debt_consolidation    0
purpose_educational           0
purpose_home_improvement      0
purpose_house                 0
purpose_major_purchase        0
purpose_medical               0
purpose_moving                0
purpose_other                 0
purpose_renewable_energy      0
purpose_

In [6]:
# 필요 없는 부분 제거
df.drop(columns = 'Unnamed: 0', inplace = True)

In [7]:
df

,installment,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,pub_rec,pub_rec_bankruptcies,annual_inc,dti,funded_amnt_inv,grade,int_rate,loan_amnt,mort_acc,num_accts_ever_120_pd,issue_d_year,issue_d_month,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 60 months,addr_state_AL,addr_state_AR,addr_state_AZ,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_IA,addr_state_ID,addr_state_IL,addr_state_IN,addr_state_KS,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_ND,addr_state_NE,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,fico_range_mean,target
0,680.82,126.0,277.0,0.0,0.0,110000.0,34.70,23200.000000,3,24.99,23200.0,2.0,0.0,2016,7,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,672.0,1
1,494.55,134.0,400.0,0.0,0.0,65000.0,17.74,16000.000000,7,7.07,16000.0,3.0,0.0,2017,10,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,822.0,1
2,146.10,226.0,130.0,2.0,2.0,50000.0,6.99,4500.000000,6,10.42,4500.0,1.0,0.0,2017,5,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,702.0,0
3,645.25,149.0,190.0,0.0,0.0,60000.0,28.94,20000.000000,6,9.99,20000.0,1.0,5.0,2015,9,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,677.0,0
4,482.17,142.0,206.0,0.0,0.0,97523.0,10.31,18999.739941,4,17.97,19000.0,3.0,0.0,2019,5,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,737.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1071205,662.45,232.0,144.0,0.0,0.0,60000.0,20.02,26450.000000,5,17.27,26500.0,1.0,0.0,2013,2,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,697.0,0
1071206,133.78,151.0,36.0,0.0,0.0,33800.0,19.18,3600.000000,3,19.99,3600.0,1.0,0.0,2014,3,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,667.0,1
1071207,583.89,125.0,216.0,0.0,0.0,71000.0,23.92,18875.000000,7,6.68,19000.0,5.0,0.0,2015,4,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,737.0,0
1071208,329.65,25.0,99.0,0.0,0.0,44000.0,12.85,9600.000000,5,14.33,9600.0,1.0,0.0,2013,5,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,692.0,0


In [8]:
# 우선 랜덤하게 data_split하고 돌려봄 6:2:2
df_train, df_test = train_test_split(df, test_size = .2, train_size = .8, random_state = 42)
df_train, df_validation = train_test_split(df_train, test_size = .25, train_size = 0.75, random_state = 42)

In [9]:
def show_time(diff):
   m, s = divmod(diff, 60)
   h, m = divmod(m, 60)
   s,m,h = int(round(s, 0)), int(round(m, 0)), int(round(h, 0))
   print("Execution Time: " + "{0:02d}:{1:02d}:{2:02d}".format(h, m, s))

# Takes in a classifier, calculates the training + prediction times and accuracy score, returns a model
def Train(clf, X, y, X_predict, y_predict, type='classification'):
    # Train
    start = time.time()
    model = clf.fit(X,y)
    end = time.time()
    print('Training time: ')
    show_time(end - start)
    training_times.append(end - start)

    # Predict
    start = time.time()
    if(type=='classification'):
        scores.append(f1_score(y_predict, model.predict(X_predict)))
    else:
        scores.append(mean_squared_error(y_test, model.predict(X_test)))
    end = time.time()
    prediction_times.append(end - start)
    print('\nPrediction time: ')
    show_time(end - start)
    return model


# Takes in model scores and plots them on a bar graph
def plot_metric(model_scores, score='Accuracy'):
    # Set figure size
    rcParams['figure.figsize'] = 20,15
    plt.bar(model_scores['Model'], height=model_scores[score])
    xlocs, xlabs = plt.xticks()
    xlocs=[i for i in range(0,17)]
    xlabs=[i for i in range(0,17)]
    if(score != 'Prediction Times'):
        for i, v in enumerate(model_scores[score]):
            plt.text(xlocs[i] - 0.25, v + 0.01, str(v))
    plt.xlabel('Model')
    plt.ylabel(score)
    plt.xticks(rotation=45)
    plt.show()


In [10]:
x_data = list(df.columns)
x_data.remove('target')

In [11]:
x_data

['installment',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'pub_rec',
 'pub_rec_bankruptcies',
 'annual_inc',
 'dti',
 'funded_amnt_inv',
 'grade',
 'int_rate',
 'loan_amnt',
 'mort_acc',
 'num_accts_ever_120_pd',
 'issue_d_year',
 'issue_d_month',
 'home_ownership_MORTGAGE',
 'home_ownership_NONE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_educational',
 'purpose_home_improvement',
 'purpose_house',
 'purpose_major_purchase',
 'purpose_medical',
 'purpose_moving',
 'purpose_other',
 'purpose_renewable_energy',
 'purpose_small_business',
 'purpose_vacation',
 'purpose_wedding',
 'term_ 60 months',
 'addr_state_AL',
 'addr_state_AR',
 'addr_state_AZ',
 'addr_state_CA',
 'addr_state_CO',
 'addr_state_CT',
 'addr_state_DC',
 'addr_state_DE',
 'addr_state_FL',
 'addr_state_GA',
 'addr_state_HI',
 'addr_state_IA',
 'addr_state_ID',
 'addr_state_IL',
 'addr_state_IN',
 'addr_state_KS',
 'addr

In [12]:
X_train = df_train[x_data]
y_train = df_train['target']

X_test = df_validation[x_data]
y_test = df_validation['target']
# Target은 Survived

In [13]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [14]:
prediction_times = []
training_times = []
scores = []
# training_times
# prediction_times

xgboost = Train(XGBClassifier(n_estimators=50, max_depth=5), X_train, y_train, X_test, y_test)
lgb = Train(LGBMClassifier(n_estimators=50, max_depth=5, num_leaves = 50), X_train, y_train, X_test, y_test)
cat = Train(CatBoostClassifier(n_estimators=50, verbose=False, max_depth=5), X_train, y_train, X_test, y_test)
#ngb = Train(NGBClassifier(n_estimators=50, verbose=False, natural_gradient=False), X_train, y_train, X_test, y_test)

Training time: 
Execution Time: 00:00:38

Prediction time: 
Execution Time: 00:00:02
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 126375, number of negative: 516351
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.168445 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2331
[LightGBM] [Info] Number of data points in the train set: 642726, number of used features: 84
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.196623 -> initscore=-1.407533
[LightGBM] [Info] Start training from score -1.407533
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [15]:
models = [
    ('XGBoost', xgboost),
    ('LightGBM', lgb),
    ('CatBoost', cat)
    #('NgBoost', ngb),
]

In [16]:
model = [xgboost, lgb, cat]
threshold = np.linspace(25, 75,51)/100

In [17]:
threshold

array([0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33, 0.34, 0.35,
       0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44, 0.45, 0.46,
       0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57,
       0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68,
       0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75])

In [38]:
df_cal = pd.DataFrame()

In [40]:
diff = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/model/total.csv')

In [41]:
diff

,Unnamed: 0,total_pymnt
0,0,16430.860000
1,1,8599.680000
2,2,4204.260000
3,3,11804.280000
4,4,5134.174383
...,...,...
1534131,1534131,469.070000
1534132,1534132,20620.112719
1534133,1534133,11793.580010
1534134,1534134,25156.048068


In [44]:
df_cal['compare_basis'] = df['target'] # 비교할 대상
df_cal['repay'] = diff['total_pymnt'] #상환액

In [45]:
# 원금
df_cal['funded'] = df['funded_amnt_inv']

In [46]:
# 상환액 - 원금
df_cal['diff'] = df_cal['repay'] - df_cal['funded']

In [47]:
# 테스트셋에 대한 예측 확률 가져오기
y_pred_proba = xgboost.predict_proba(df[x_data])[:, 1]  # 클래스 1에 대한 예측 확률 가져오기
# 사용자가 정의한 임계값으로 예측을 바꾸기


for cut in threshold:
    y_pred = (y_pred_proba >= cut).astype(int)
    df_cal[f"xgb{cut}"] = y_pred
    # 딕셔너리에 저장



In [48]:
# 테스트셋에 대한 예측 확률 가져오기
y_pred_proba = lgb.predict_proba(df[x_data])[:, 1]  # 클래스 1에 대한 예측 확률 가져오기
# 사용자가 정의한 임계값으로 예측을 바꾸기


for cut in threshold:
    y_pred = (y_pred_proba >= cut).astype(int)
    df_cal[f"lgb{cut}"] = y_pred
    # 딕셔너리에 저장



In [49]:
# 테스트셋에 대한 예측 확률 가져오기
y_pred_proba = cat.predict_proba(df[x_data])[:, 1]  # 클래스 1에 대한 예측 확률 가져오기
# 사용자가 정의한 임계값으로 예측을 바꾸기


for cut in threshold:
    y_pred = (y_pred_proba >= cut).astype(int)
    df_cal[f"cat{cut}"] = y_pred
    # 딕셔너리에 저장



In [50]:
df_cal

,compare_basis,funded,repay,diff,xgb0.25,xgb0.26,xgb0.27,xgb0.28,xgb0.29,xgb0.3,xgb0.31,xgb0.32,xgb0.33,xgb0.34,xgb0.35,xgb0.36,xgb0.37,xgb0.38,xgb0.39,xgb0.4,xgb0.41,xgb0.42,xgb0.43,xgb0.44,xgb0.45,xgb0.46,xgb0.47,xgb0.48,xgb0.49,xgb0.5,xgb0.51,xgb0.52,xgb0.53,xgb0.54,xgb0.55,xgb0.56,xgb0.57,xgb0.58,xgb0.59,xgb0.6,xgb0.61,xgb0.62,xgb0.63,xgb0.64,xgb0.65,xgb0.66,xgb0.67,xgb0.68,xgb0.69,xgb0.7,xgb0.71,xgb0.72,xgb0.73,xgb0.74,xgb0.75,lgb0.25,lgb0.26,lgb0.27,lgb0.28,lgb0.29,lgb0.3,lgb0.31,lgb0.32,lgb0.33,lgb0.34,lgb0.35,lgb0.36,lgb0.37,lgb0.38,lgb0.39,lgb0.4,lgb0.41,lgb0.42,lgb0.43,lgb0.44,lgb0.45,lgb0.46,lgb0.47,lgb0.48,lgb0.49,lgb0.5,lgb0.51,lgb0.52,lgb0.53,lgb0.54,lgb0.55,lgb0.56,lgb0.57,lgb0.58,lgb0.59,lgb0.6,lgb0.61,lgb0.62,lgb0.63,lgb0.64,lgb0.65,lgb0.66,lgb0.67,lgb0.68,lgb0.69,lgb0.7,lgb0.71,lgb0.72,lgb0.73,lgb0.74,lgb0.75,cat0.25,cat0.26,cat0.27,cat0.28,cat0.29,cat0.3,cat0.31,cat0.32,cat0.33,cat0.34,cat0.35,cat0.36,cat0.37,cat0.38,cat0.39,cat0.4,cat0.41,cat0.42,cat0.43,cat0.44,cat0.45,cat0.46,cat0.47,cat0.48,cat0.49,cat0.5,cat0.51,cat0.52,cat0.53,cat0.54,cat0.55,cat0.56,cat0.57,cat0.58,cat0.59,cat0.6,cat0.61,cat0.62,cat0.63,cat0.64,cat0.65,cat0.66,cat0.67,cat0.68,cat0.69,cat0.7,cat0.71,cat0.72,cat0.73,cat0.74,cat0.75
0,1,23200.000000,16430.860000,-6769.140000,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,16000.000000,8599.680000,-7400.320000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,4500.000000,4204.260000,-295.740000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,20000.000000,11804.280000,-8195.720000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,18999.739941,5134.174383,-13865.565557,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1071205,0,26450.000000,23445.384893,-3004.615107,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1071206,1,3600.000000,42896.44

In [51]:
df_cal.columns

Index(['compare_basis', 'funded', 'repay', 'diff', 'xgb0.25', 'xgb0.26',
       'xgb0.27', 'xgb0.28', 'xgb0.29', 'xgb0.3', 'xgb0.31', 'xgb0.32',
       'xgb0.33', 'xgb0.34', 'xgb0.35', 'xgb0.36', 'xgb0.37', 'xgb0.38',
       'xgb0.39', 'xgb0.4', 'xgb0.41', 'xgb0.42', 'xgb0.43', 'xgb0.44',
       'xgb0.45', 'xgb0.46', 'xgb0.47', 'xgb0.48', 'xgb0.49', 'xgb0.5',
       'xgb0.51', 'xgb0.52', 'xgb0.53', 'xgb0.54', 'xgb0.55', 'xgb0.56',
       'xgb0.57', 'xgb0.58', 'xgb0.59', 'xgb0.6', 'xgb0.61', 'xgb0.62',
       'xgb0.63', 'xgb0.64', 'xgb0.65', 'xgb0.66', 'xgb0.67', 'xgb0.68',
       'xgb0.69', 'xgb0.7', 'xgb0.71', 'xgb0.72', 'xgb0.73', 'xgb0.74',
       'xgb0.75', 'lgb0.25', 'lgb0.26', 'lgb0.27', 'lgb0.28', 'lgb0.29',
       'lgb0.3', 'lgb0.31', 'lgb0.32', 'lgb0.33', 'lgb0.34', 'lgb0.35',
       'lgb0.36', 'lgb0.37', 'lgb0.38', 'lgb0.39', 'lgb0.4', 'lgb0.41',
       'lgb0.42', 'lgb0.43', 'lgb0.44', 'lgb0.45', 'lgb0.46', 'lgb0.47',
       'lgb0.48', 'lgb0.49', 'lgb0.5', 'lgb0.51', 'lgb0.52

In [52]:
compare_list = ['xgb0.25', 'xgb0.26',
       'xgb0.27', 'xgb0.28', 'xgb0.29', 'xgb0.3', 'xgb0.31', 'xgb0.32',
       'xgb0.33', 'xgb0.34', 'xgb0.35', 'xgb0.36', 'xgb0.37', 'xgb0.38',
       'xgb0.39', 'xgb0.4', 'xgb0.41', 'xgb0.42', 'xgb0.43', 'xgb0.44',
       'xgb0.45', 'xgb0.46', 'xgb0.47', 'xgb0.48', 'xgb0.49', 'xgb0.5',
       'xgb0.51', 'xgb0.52', 'xgb0.53', 'xgb0.54', 'xgb0.55', 'xgb0.56',
       'xgb0.57', 'xgb0.58', 'xgb0.59', 'xgb0.6', 'xgb0.61', 'xgb0.62',
       'xgb0.63', 'xgb0.64', 'xgb0.65', 'xgb0.66', 'xgb0.67', 'xgb0.68',
       'xgb0.69', 'xgb0.7', 'xgb0.71', 'xgb0.72', 'xgb0.73', 'xgb0.74',
       'xgb0.75', 'lgb0.25', 'lgb0.26', 'lgb0.27', 'lgb0.28', 'lgb0.29',
       'lgb0.3', 'lgb0.31', 'lgb0.32', 'lgb0.33', 'lgb0.34', 'lgb0.35',
       'lgb0.36', 'lgb0.37', 'lgb0.38', 'lgb0.39', 'lgb0.4', 'lgb0.41',
       'lgb0.42', 'lgb0.43', 'lgb0.44', 'lgb0.45', 'lgb0.46', 'lgb0.47',
       'lgb0.48', 'lgb0.49', 'lgb0.5', 'lgb0.51', 'lgb0.52', 'lgb0.53',
       'lgb0.54', 'lgb0.55', 'lgb0.56', 'lgb0.57', 'lgb0.58', 'lgb0.59',
       'lgb0.6', 'lgb0.61', 'lgb0.62', 'lgb0.63', 'lgb0.64', 'lgb0.65',
       'lgb0.66', 'lgb0.67', 'lgb0.68', 'lgb0.69', 'lgb0.7', 'lgb0.71',
       'lgb0.72', 'lgb0.73', 'lgb0.74', 'lgb0.75', 'cat0.25', 'cat0.26',
       'cat0.27', 'cat0.28', 'cat0.29', 'cat0.3', 'cat0.31', 'cat0.32',
       'cat0.33', 'cat0.34', 'cat0.35', 'cat0.36', 'cat0.37', 'cat0.38',
       'cat0.39', 'cat0.4', 'cat0.41', 'cat0.42', 'cat0.43', 'cat0.44',
       'cat0.45', 'cat0.46', 'cat0.47', 'cat0.48', 'cat0.49', 'cat0.5',
       'cat0.51', 'cat0.52', 'cat0.53', 'cat0.54', 'cat0.55', 'cat0.56',
       'cat0.57', 'cat0.58', 'cat0.59', 'cat0.6', 'cat0.61', 'cat0.62',
       'cat0.63', 'cat0.64', 'cat0.65', 'cat0.66', 'cat0.67', 'cat0.68',
       'cat0.69', 'cat0.7', 'cat0.71', 'cat0.72', 'cat0.73', 'cat0.74',
       'cat0.75']

In [53]:
for com in compare_list:
    TN_amount = ((df_cal['compare_basis'] == 0) & (df_cal[com] == 0)) * df_cal['diff']
    TP_amount = ((df_cal['compare_basis'] == 1) & (df_cal[com] == 1)) * df_cal['diff']
    FN_amount = ((df_cal['compare_basis'] == 1) & (df_cal[com] == 0)) * df_cal['diff']


    # 총 미상환액 계산
    total_unpaid_amount = TN_amount.sum() - TP_amount.sum() - FN_amount.sum()

    # 결과 출력
    print("Total Unpaid Amount:", com, total_unpaid_amount)

Total Unpaid Amount: xgb0.25 318674420.9512275
Total Unpaid Amount: xgb0.26 282581546.07661974
Total Unpaid Amount: xgb0.27 247649024.5733624
Total Unpaid Amount: xgb0.28 209593978.45931172
Total Unpaid Amount: xgb0.29 170199972.43752593
Total Unpaid Amount: xgb0.3 131701203.42859207
Total Unpaid Amount: xgb0.31 91817519.90546042
Total Unpaid Amount: xgb0.32 52042568.95681146
Total Unpaid Amount: xgb0.33 13135374.852144897
Total Unpaid Amount: xgb0.34 -21662489.45887667
Total Unpaid Amount: xgb0.35 -59123862.831619084
Total Unpaid Amount: xgb0.36 -93004983.6825378
Total Unpaid Amount: xgb0.37 -126075996.69986302
Total Unpaid Amount: xgb0.38 -154035826.71270794
Total Unpaid Amount: xgb0.39 -183180221.08863485
Total Unpaid Amount: xgb0.4 -209574184.1650107
Total Unpaid Amount: xgb0.41 -235444791.42087358
Total Unpaid Amount: xgb0.42 -258587415.04776788
Total Unpaid Amount: xgb0.43 -282138885.94658005
Total Unpaid Amount: xgb0.44 -303674533.24176455
Total Unpaid Amount: xgb0.45 -324941405